<a href="https://colab.research.google.com/github/niteshctrl/fake_news_classifier/blob/main/4_Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set Up Kaggle API


from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nitesh1612","key":"172fe3df80e3530007ceee0d567a2580"}'}

In [ ]:
# Install Kaggle Library and auth permission


!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [ ]:
# Data Download from Kaggle


! kaggle competitions download -c 'fake-news' --path '/content/data'

 53% 5.00M/9.42M [00:00<00:00, 14.0MB/s]
100% 9.42M/9.42M [00:00<00:00, 23.8MB/s]
 89% 33.0M/37.0M [00:00<00:00, 25.1MB/s]
100% 37.0M/37.0M [00:00<00:00, 42.4MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 42.8MB/s]


In [ ]:
# Unzip the files


!unzip /content/data/test.csv.zip -d /content/data
!unzip /content/data/train.csv.zip -d /content/data

Archive:  /content/data/test.csv.zip
  inflating: /content/data/test.csv  
Archive:  /content/data/train.csv.zip
  inflating: /content/data/train.csv  


In [ ]:
# Import libraries

import re
import nltk
import pandas as pd
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from keras import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from nltk.stem.porter import PorterStemmer
from keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Read the CSV's into dataframe


train = pd.read_csv('/content/data/train.csv')
test = pd.read_csv('/content/data/test.csv')

In [ ]:
train.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [ ]:
test.tail()

,id,title,author,text
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...
5199,25999,"Awkward Sex, Onscreen and Off - The New York T...",Teddy Wayne,Perhaps you’ve seen the new TV series whose pi...


In [ ]:
# Check NaN Values


print(train['title'].isnull().value_counts())
print(test['title'].isnull().value_counts())

False    20242
True       558
Name: title, dtype: int64
False    5078
True      122
Name: title, dtype: int64


In [ ]:
# Check Texts whose title is NULL


train[train['title'].isnull()]['text']

53       Sounds like he has our president pegged. What ...
120      Same people all the time , i dont know how you...
124      You know, outside of any morality arguments, i...
140      There is a lot more than meets the eye to this...
196                 They got the heater turned up on high.
                               ...                        
20568    Amusing comment Gary! “Those   week!” So, are ...
20627    No she doesn’t have more money than God, every...
20636                                   Trump all the way!
20771    DYN's Statement on Last Week's Botnet Attack h...
20772    Kinda reminds me of when Carter gave away the ...
Name: text, Length: 558, dtype: object

In [ ]:
# Drop NaN values from train data

train = train.dropna()

In [ ]:
print(train.isnull().value_counts())

id     title  author  text   label
False  False  False   False  False    18285
dtype: int64


In [ ]:
# Reset Index of train data


train = train.reset_index()
train

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
# Drop Index and id columns


train = train.drop(['index','id'], axis=1)
train

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
18280,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


# Preprocessing 

1. Stopword Removal
2. Steming
3. Symbols Removal
4. All to Lowercase

In [ ]:
porter = PorterStemmer()
corpus = []

In [ ]:
# Preprocessing steps


for i in tqdm(range(len(train))):
    title = re.sub('[^a-zA-Z]', ' ', str(train.title[i])) # Remove every character except English alphabets
    title = title.lower() # Convert to lowercase
    title = title.split()

    title_prep_words = []
    title_string = ''
    for word in title:
        if word not in stopwords.words('english'): # Remove Stopwords
            title_prep_words.append(porter.stem(word)) # Stem the word and append it to list
    title_string = ' '.join(title_prep_words)
    corpus.append(title_string)

In [ ]:
# Define Vocabulary Size

vocab_size = 10000
embedding_vec_features = 50

In [ ]:
# One-hot representation of sentences in the corpus


one_hot_rep = [one_hot(sentence, vocab_size) for sentence in corpus]

In [ ]:
# Embedding Representation


embedded_docs = pad_sequences(one_hot_rep)

In [ ]:
# Build the Model


model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_vec_features))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          500000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 560,501
Trainable params: 560,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Train Model

model.fit(embedded_docs, train.label, epochs=10, validation_split=0.2)

Epoch 1/10
458/458 [==============================] - 38s 12ms/step - loss: 0.3695 - accuracy: 0.8159 - val_loss: 0.1911 - val_accuracy: 0.9210
Epoch 2/10
458/458 [==============================] - 5s 10ms/step - loss: 0.1251 - accuracy: 0.9504 - val_loss: 0.1946 - val_accuracy: 0.9223
Epoch 3/10
458/458 [==============================] - 5s 10ms/step - loss: 0.0692 - accuracy: 0.9765 - val_loss: 0.2234 - val_accuracy: 0.9174
Epoch 4/10
458/458 [==============================] - 5s 11ms/step - loss: 0.0356 - accuracy: 0.9891 - val_loss: 0.2542 - val_accuracy: 0.9207
Epoch 5/10
458/458 [==============================] - 5s 11ms/step - loss: 0.0200 - accuracy: 0.9937 - val_loss: 0.3408 - val_accuracy: 0.9155
Epoch 6/10
458/458 [==============================] - 5s 11ms/step - loss: 0.0115 - accuracy: 0.9965 - val_loss: 0.3876 - val_accuracy: 0.9166
Epoch 7/10
458/458 [==============================] - 5s 11ms/step - loss: 0.0091 - accuracy: 0.9972 - val_loss: 0.4095 - val_accuracy: 0.915

In [ ]:
# Bidirectional LSTM

model_b = Sequential()
model_b.add(Embedding(input_dim=vocab_size, output_dim=embedding_vec_features))
model_b.add(Bidirectional(LSTM(100)))
model_b.add(Dense(1, activation='sigmoid'))

model_b.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_b.fit(embedded_docs, train.label, epochs=10, validation_split=0.2)

Epoch 1/10
458/458 [==============================] - 49s 37ms/step - loss: 0.3824 - accuracy: 0.7985 - val_loss: 0.1800 - val_accuracy: 0.9256
Epoch 2/10
458/458 [==============================] - 15s 34ms/step - loss: 0.1051 - accuracy: 0.9604 - val_loss: 0.2024 - val_accuracy: 0.9275
Epoch 3/10
458/458 [==============================] - 15s 34ms/step - loss: 0.0624 - accuracy: 0.9792 - val_loss: 0.2329 - val_accuracy: 0.9210
Epoch 4/10
458/458 [==============================] - 16s 34ms/step - loss: 0.0351 - accuracy: 0.9887 - val_loss: 0.2735 - val_accuracy: 0.9161
Epoch 5/10
458/458 [==============================] - 15s 34ms/step - loss: 0.0233 - accuracy: 0.9935 - val_loss: 0.3650 - val_accuracy: 0.9098
Epoch 6/10
458/458 [==============================] - 15s 33ms/step - loss: 0.0122 - accuracy: 0.9964 - val_loss: 0.4036 - val_accuracy: 0.9111
Epoch 7/10
458/458 [==============================] - 15s 33ms/step - loss: 0.0105 - accuracy: 0.9973 - val_loss: 0.4208 - val_accuracy:

# References

https://www.kaggle.com/c/fake-news/overview